In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
df = pd.read_csv('./data/train_1.csv')
df.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [6]:
tdata = df[df.columns[1:]].values
print(tdata.shape)

(145063, 550)


In [7]:
class CSVDataset(Dataset):

    def __init__(self,file_path, n_x, n_y):
        super(CSVDataset,self).__init__()
        self.n_x = n_x
        self.n_y = n_y
        self.data = pd.read_csv(file_path,nrows=100)
        self.data = self.data.fillna(0)
        self.data = self.data[self.data.columns[1:]].values

    def __len__(self):
        return (self.data.shape[0]-self.n_x-self.n_y)*self.data.shape[1]

    def __getitem__(self, index):
        line = index //  (self.data.shape[1]-self.n_x-self.n_y)
        i = self.n_x + self.n_y + (index % (self.data.shape[1]-self.n_x-self.n_y))

        item = (torch.FloatTensor(self.data[line,i-self.n_y-self.n_x:i-self.n_y]),
                torch.FloatTensor(self.data[line,i-self.n_y:i]))
        item = (torch.stack([item[0]]),torch.stack([item[1]]))
        return item

In [8]:
n_x = 50
n_y = 10
dataset = CSVDataset('./data/train_1.csv', n_x, n_y)

In [9]:
dataloader = DataLoader(dataset, batch_size=256)

In [10]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size,device=device),
                            torch.zeros(1,1,self.hidden_layer_size,device=device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), (self.hidden_cell[0],self.hidden_cell[1]))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [11]:
model = LSTM(input_size = n_x, output_size = n_y).to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
print(model)

LSTM(
  (lstm): LSTM(50, 100)
  (linear): Linear(in_features=100, out_features=10, bias=True)
)


In [13]:
epochs = 5

for i in range(epochs):
    for i_batch, data in enumerate(dataloader):
        x = data[0].to(device)
        y = data[1].to(device)
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size,device=device),
                        torch.zeros(1, 1, model.hidden_layer_size,device=device))

        y_pred = model(x)

        single_loss = loss_function(y_pred, y)
        single_loss.backward()
        optimizer.step()
        
        if i_batch%100 == 0:
            print(f'Loss at {i_batch}')
        
    if i%1 == 0:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')
    

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

/home/clock/anaconda3/envs/my_env/lib/python3.8/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([256, 1, 10])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Loss at 0


/home/clock/anaconda3/envs/my_env/lib/python3.8/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([240, 1, 10])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch:   0 loss: 1152.16857910
Loss at 0
epoch:   1 loss: 906.67822266
Loss at 0
epoch:   2 loss: 758.67199707
Loss at 0
epoch:   3 loss: 686.96795654
Loss at 0
epoch:   4 loss: 652.62060547
epoch:   4 loss: 652.6206054688
